In [10]:
from twelvedata import TDClient
import time


api_key = "ced60696328f4b6a8dbdf54033a32bb5"
td = TDClient(api_key)

# Define symbols list
symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'FB', 'NVDA', 'NFLX', 'PYPL', 'INTC']

# Define SMA window sizes
sma_window_sizes = [5, 8, 13, 20]

def get_sma(symbol, window_size):
    data = td.time_series(symbol=symbol, interval='1min', outputsize=window_size)
    sma = sum(data['close']) / window_size
    return sma

def check_rules(symbol):
    last_sma_values = []
    previous_sma_values = []
    for window_size in sma_window_sizes:
        last_sma_values.append(get_sma(symbol, window_size))
        previous_sma_values.append(get_sma(symbol, window_size + 1))
    
    last_sma5, last_sma8, last_sma13, last_sma20 = last_sma_values
    prev_sma5, prev_sma8, prev_sma13, prev_sma20 = previous_sma_values

    if (last_sma5 > last_sma8) and (prev_sma5 < prev_sma8) and (last_sma5 > last_sma20):
        return 'Buy'
    elif (last_sma5 < last_sma8) and (prev_sma5 > prev_sma8) and (last_sma8 > last_sma13):
        return 'Sell'
    else:
        return 'None'

while True:
    current_time = time.localtime()
    if current_time.tm_hour >= 9 and current_time.tm_hour < 16:
        for symbol in symbols:
            action = check_rules(symbol)
            if action == 'Buy':
                print(f'Buy {symbol} at {time.strftime("%Y-%m-%d %H:%M:%S", current_time)} for {get_sma(symbol, 1)}')
            elif action == 'Sell':
                print(f'Sell {symbol} at {time.strftime("%Y-%m-%d %H:%M:%S", current_time)} for {get_sma(symbol, 1)}')
            else:
                print(f'Do nothing for {symbol}')
    else:
        print('Market closed. Waiting for it to open...')
        time.sleep(60)  # Check every minute if market has opened